In [1]:
from sqlalchemy import create_engine, inspect
import pandas as pd

In [2]:
engine = create_engine("sqlite:///../db/mlsseasons.sqlite", echo=False)

In [3]:
inspect(engine)

In [4]:
dataset=pd.DataFrame(engine.execute("""
select s."First Name", s."Last Name", s.Position,
c.Club, c.pts, c."#" as standing, c.season
from seasons c
join club_map m on m.long_name = c.Club
join salaries s on s."Club (grouped)" = m.short_name
where c.season = s.Season
"""),columns=['First Name','Last Name','Position','Club','PTS','Standing','Season'])
dataset

,First Name,Last Name,Position,Club,PTS,Standing,Season
0,None,Carreiro da Silva Fred,M-F,DCD.C. United,55,1,2007
1,Ben,Olsen,M,DCD.C. United,55,1,2007
2,Brad,North,F,DCD.C. United,55,1,2007
3,Brian,Carroll,M,DCD.C. United,55,1,2007
4,Bryan,Arguez,M-D,DCD.C. United,55,1,2007
...,...,...,...,...,...,...,...
6206,Quincy,Amarikwa,F,SJSan Jose Earthquakes,21,23,2018
6207,Shea,Salinas,M,SJSan Jose Earthquakes,21,23,2018
6208,Tommy,Thompson,M,SJSan Jose Earthquakes,21,23,2018
6209,"Valeri ""Vako""",Qazaishvili,M,SJSan Jose Earthquakes,21,23,2018


In [5]:
dataset["First Name"] = dataset["First Name"].fillna("")

In [6]:
dataset['Player'] = dataset['First Name'] + dataset['Last Name'] # + dataset['Position']
dataset['Team'] = dataset['Club'] + dataset['Season'].apply(str)
dataset

,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team
0,,Carreiro da Silva Fred,M-F,DCD.C. United,55,1,2007,Carreiro da Silva Fred,DCD.C. United2007
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007
2,Brad,North,F,DCD.C. United,55,1,2007,BradNorth,DCD.C. United2007
3,Brian,Carroll,M,DCD.C. United,55,1,2007,BrianCarroll,DCD.C. United2007
4,Bryan,Arguez,M-D,DCD.C. United,55,1,2007,BryanArguez,DCD.C. United2007
...,...,...,...,...,...,...,...,...,...
6206,Quincy,Amarikwa,F,SJSan Jose Earthquakes,21,23,2018,QuincyAmarikwa,SJSan Jose Earthquakes2018
6207,Shea,Salinas,M,SJSan Jose Earthquakes,21,23,2018,SheaSalinas,SJSan Jose Earthquakes2018
6208,Tommy,Thompson,M,SJSan Jose Earthquakes,21,23,2018,TommyThompson,SJSan Jose Earthquakes2018
6209,"Valeri ""Vako""",Qazaishvili,M,SJSan Jose Earthquakes,21,23,2018,"Valeri ""Vako""Qazaishvili",SJSan Jose Earthquakes2018


In [7]:
a.keys()

NameError: name 'a' is not defined

In [8]:
dataset.groupby("Player").count()["Season"].describe()

count    2250.000000
mean        2.758667
std         2.360747
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        12.000000
Name: Season, dtype: float64

In [9]:
y = dataset["Player"].tolist()

In [10]:
 from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y

array([ 384,  231,  279, ..., 2122, 2165, 2222], dtype=int64)

In [11]:
dataset['encoded_player'] = encoded_y
dataset['present'] = 1.0
dataset.loc[dataset['Player'] == 'BenOlsen']

,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team,encoded_player,present
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007,231,1.0
627,Ben,Olsen,M,DCD.C. United,38,10,2008,BenOlsen,DCD.C. United2008,231,1.0
1005,Ben,Olsen,M,DCD.C. United,40,10,2009,BenOlsen,DCD.C. United2009,231,1.0


In [12]:
samples = dataset.pivot(index="Team", columns="encoded_player", values='present')
samples

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
Team,,,,,,,,,,,,,,,,,,,,,
ATLAtlanta United FC2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATLAtlanta United FC2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VANVancouver Whitecaps FC2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
samples.describe()

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
count,2.0,10.0,4.0,1.0,2.0,1.0,4.0,3.0,1.0,5.0,...,2.0,1.0,2.0,6.0,1.0,1.0,1.0,2.0,2.0,4.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [15]:
dataset['Standing'] = dataset['Standing'].astype(int)
dataset['PTS'] = dataset['PTS'].astype(int)
y = dataset.groupby('Team').mean()['Standing'].sort_index()
y

Team
ATLAtlanta United FC2017          4
ATLAtlanta United FC2018          2
CHIChicago Fire FC2007            8
CHIChicago Fire FC2008            3
CHIChicago Fire FC2009            6
                                 ..
VANVancouver Whitecaps FC2014     9
VANVancouver Whitecaps FC2015     3
VANVancouver Whitecaps FC2016    16
VANVancouver Whitecaps FC2017     8
VANVancouver Whitecaps FC2018    14
Name: Standing, Length: 218, dtype: int32

In [16]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets

In [17]:
X = samples.fillna(0).sort_index()

In [18]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [19]:
X_test.shape

(55, 2251)

In [20]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=X_train.shape[-1]))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                72064     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 72,737
Trainable params: 72,737
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

In [22]:
# Fit the model to the training data
model.fit(
    X_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
6/6 - 0s - loss: 122.7734 - mean_absolute_error: 9.4394
Epoch 2/100
6/6 - 0s - loss: 119.7668 - mean_absolute_error: 9.2849
Epoch 3/100
6/6 - 0s - loss: 116.7018 - mean_absolute_error: 9.1175
Epoch 4/100
6/6 - 0s - loss: 112.6820 - mean_absolute_error: 8.8956
Epoch 5/100
6/6 - 0s - loss: 107.6189 - mean_absolute_error: 8.5997
Epoch 6/100
6/6 - 0s - loss: 100.6210 - mean_absolute_error: 8.2266
Epoch 7/100
6/6 - 0s - loss: 92.0785 - mean_absolute_error: 7.7564
Epoch 8/100
6/6 - 0s - loss: 81.7319 - mean_absolute_error: 7.1780
Epoch 9/100
6/6 - 0s - loss: 69.7094 - mean_absolute_error: 6.5186
Epoch 10/100
6/6 - 0s - loss: 56.7000 - mean_absolute_error: 5.8062
Epoch 11/100
6/6 - 0s - loss: 43.7446 - mean_absolute_error: 5.1210
Epoch 12/100
6/6 - 0s - loss: 32.7690 - mean_absolute_error: 4.5318
Epoch 13/100
6/6 - 0s - loss: 24.5297 - mean_absolute_error: 3.9654
Epoch 14/100
6/6 - 0s - loss: 19.6598 - mean_absolute_error: 3.6003
Epoch 15/100
6/6 - 0s - loss: 16.3054 - mean_absolu

In [23]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 32.5726 - mean_absolute_error: 4.4805
Normal Neural Network - Loss: 32.5726432800293, Accuracy: 4.480509281158447
